In [1]:
# Install dependencies (only once)
!pip install pytesseract pdf2image
!apt-get install tesseract-ocr -y
!apt-get install tesseract-ocr-ben -y
!apt-get install poppler-utils -y

from pdf2image import convert_from_path
import pytesseract

# Set path to your PDF
pdf_path = "/content/HSC26-Bangla1st-Paper.pdf"  # upload it to Colab first

# Convert PDF to images
images = convert_from_path(pdf_path, dpi=300)

# Run Bengali OCR on each page
ocr_text = ""
for i, img in enumerate(images):
    text = pytesseract.image_to_string(img, lang="ben", config="--psm 6")
    ocr_text += f"\n--- Page {i+1} ---\n{text}"

# Save to a text file
with open("bangla_ocr_output.txt", "w", encoding="utf-8") as f:
    f.write(ocr_text)

# Show some text preview
print(ocr_text[:10])


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ben
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 516 kB of archives.
After this operation, 870 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ben all 1:4.00~git30-7274cfa-1.1 [516 kB]
Fetched 516 kB in 1s (503 kB/s)
Selecting previously unselected package tesseract-ocr-ben.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-ben_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-ben (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-ben 

In [2]:
!pip install pymupdf langchain
!pip install -U langchain langchain-community pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.71
    Uninstalling langchain-core-0.3.71:
      Successfully uninstalled langchain-core-0.3.71
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.26
    Uninstalling langchain-0.3.26:


In [12]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔹 Step 1: Load Text
loader = TextLoader("/content/bangla_ocr_output.txt", encoding="utf-8")
docs = loader.load()

# 🔹 Step 2: Chunk
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)



In [17]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

# Step 1: Load embedding model
embed_model = SentenceTransformer("sentence-transformers/LaBSE")

#  Step 2: Encode chunks
chunk_texts = [doc.page_content for doc in chunks]
chunk_embeddings = embed_model.encode(chunk_texts, show_progress_bar=True)

#  Step 3: Build FAISS Index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

#  Step 4: Load QA Model (Multilingual)
qa_pipeline = pipeline(
    "question-answering",
    model="deepset/xlm-roberta-large-squad2",
    tokenizer="deepset/xlm-roberta-large-squad2"
)

# Step 5: Define Retrieval + QA Function
def answer_question(query, top_k=3, show_context=False):
    query_embedding = embed_model.encode([query])
    D, I = index.search(np.array(query_embedding), top_k)
    retrieved_chunks = [chunk_texts[i] for i in I[0]]
    context = " ".join(retrieved_chunks)

    if show_context:
        print("\n[Retrieved Context Preview]\n", context[:500], "\n---")

    result = qa_pipeline(question=query, context=context)
    return result["answer"]

# Step 6: Ask a Question
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
answer = answer_question(query, show_context=True)
print("উত্তর:", answer)


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



[Retrieved Context Preview]
 ১৭। অনুপমের শিক্ষাগত যোগ্যতা কি?
(ক) বিএ পাশ (খ) এমএ পাশ (গ) বিএসসি পাশ (ঘ) এমএসসি পাশ
১৮। 'মেয়ে যদি বলো, তবে' উক্তিটি কার?
(ক) অনুপমের (খ) হরিশের (গ) শস্তুনাথের (ঘ) মামার
১৯। 'অপরিচিতা' গল্পে রসিক মনের মানুষ কে?
(ক) অনুপম (খ) ঘটক (গ) হরিশ (ঘ) মামা
২০। 'একবার মামার কাছে কথাটা পাড়িয়া দেখ' উক্তিটি কার?
(ক) বিনুদাদার (খ) শস্তুনাথের (গ) হরিশের (ঘ) অনুপমের
২১। হরিশ কোথায় কাজ করত?
(ক) কলকাতায় (খ) আন্দামানে (গ) রাজপুরে (ঘ) কানপুরে
২২। ' এককালে ইহাদের বংশে লক্ষ্মীর মঙ্গলঘট ভরা ছিল' উক্তিটিতে কাদের  
---
উত্তর:  শস্তুনাথ


In [18]:
query = "Who was called Anupam's god of fortune?"
answer = answer_question(query, show_context=True)
print("উত্তর:", answer)


[Retrieved Context Preview]
 &০। কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
(ক) হরিশকে (খ) মামাকে (গ) বিনুদাকে (ঘ) শস্তুনাথকে
&১। কার টাকার প্রতি আসক্তি বেশি?
(ক) শস্তুনাথের (খ) কল্যাণীর (গ) অনুপমের (ঘ) মামার
&২। 'কিছুদিন পূর্বে এমএ পাশ করিয়াছি'- উক্তিটি কার?
(ক) মামার (খ) বিনুদার (গ) অনুপমের (ঘ) হরিশের
৫৩। 'একবার মামার কাছে কথাটা পাড়িয়া দেখ'- কথাটি কীসের?
(ক) দানের (খ) চাকরির (গ) বিয়ের (ঘ) ভ্রমণের
&৪। বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
(ক) ১৪ বছর (খ) ১৫ বছর (গ) ১৬ বছর (ঘ) ১৭ বছর --- Page 1 ---
10911139?
1502 
---
উত্তর:  মামাকে


In [20]:
query = "অপরিচিতা মেয়েটির সঙ্গে কতজন মেয়ে ছিল?"
answer = answer_question(query, show_context=True)
print("উত্তর:", answer)


[Retrieved Context Preview]
 (ক) ১৪/১৫ বছর (খ) ১৫/১৬ বছর (গ) ১৬/১৭ বছর (ঘ) ১৭/১৮ বছর
৪২। অপরিচিতা মেয়েটির সঙ্গে কতজন মেয়ে ছিল?
(ক) ২/৩ জন (খ) ৩/৪ জন (গ) ৪/৫ জন (ঘ) ৫/৬ জন
৪৩। কল্যাণী স্টেশন হতে কী খাবার কিনে নেয়?
(ক) চানা-মুঠ (খ) ঝালমুড়ি (গ) চিনেবাদাম (ঘ) ঝুরিভাজা
৪৪। শস্তুনাথ পেশায় কী ছিলেন?
(ক) উকিল (খ) শিক্ষক (গ) ডাক্তার (ঘ) ব্যবসায়ী
৪৫। মাতৃ-আজ্ঞা বলতে কল্যাণী কার প্রতি ইঙ্গিত করেছে?
(ক) মায়ের প্রতি (খ) মাতৃভূমির প্রতি (গ) ধরণীর প্রতি (ঘ) অন্নপূর্ণার প্রতি
৪৬। বিবাহের সময় অনুপমের বয়স কত ছিল?
(ক) ২১ বছর (খ) ২৩ ব 
---
উত্তর:  ২/৩ জন
